DATA IMPORT AND CLEANUP

In [55]:
import pandas as pd
import numpy as np
import json
from json import JSONDecodeError

df_metadata = pd.read_csv("./data/movies_metadata.csv", dtype=str)

# Create has_homepage column
df_metadata['has_homepage'] = np.where(df_metadata['homepage'].isnull(), 0, 1 )
# drop uneeded columns
df_metadata = df_metadata.drop(['homepage', 'overview', 'poster_path', 'original_title', 'status', 'tagline', 'title', 'video'], axis=1)


# Fill the belongs_to_collection values with a 0 or 1
df_metadata['belongs_to_collection'] = df_metadata['belongs_to_collection'].fillna(0)
df_metadata.loc[df_metadata['belongs_to_collection'] != 0, 'belongs_to_collection'] = 1

# Cast belongs_to_collection to int
df_metadata = df_metadata.astype({'belongs_to_collection' : int})

# Eliminate erroneous columns (where id is not an int)
df_metadata = df_metadata[df_metadata['id'].apply(lambda x: str(x).isdigit())]
# cast id to int
df_metadata = df_metadata.astype({'id' : int})


df_metadata.head()

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,has_homepage
0,False,1,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",7.7,5415,1
1,False,0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.9,2413,0
2,False,1,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",6.5,92,0
3,False,0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",6.1,34,0
4,False,1,0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",5.7,173,0


In [56]:
# Change genres column into a list of json (dict) objects 
df_metadata['genres'] = df_metadata['genres'].apply(lambda x: json.loads(x.replace('\'', '"')))

# Find all genre types
Genres = dict()
def checkGenre(listObj):
    for obj in listObj:
        if obj['name'] not in Genres:
            Genres[obj['name']] = 0
        Genres[obj['name']] += 1
    return listObj
        
     
df_metadata['genres'].apply(checkGenre)


for gen in Genres:
    df_metadata[gen] = 0

def fillGenres(row):
    for obj in row['genres']:
        row[obj['name']] = 1
    return row
    
df_metadata = df_metadata.apply(fillGenres, axis=1)

# drop genres column now that data has been flattened
df_metadata = df_metadata.drop(['genres'], axis=1)


df_metadata[['id'] + list(Genres.keys())].head()


,id,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,...,Horror,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie
0,862,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8844,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15602,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,31357,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11862,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# Change production_companies column into a list of json (dict) objects 
#clean up

ProdCompanies = dict()
def tester(val):
    try:
        compList = json.loads(val)
    except JSONDecodeError as e:
        return
       
    for entry in compList:
        if entry['name'] not in ProdCompanies:
            ProdCompanies[entry['name']] = 0
        ProdCompanies[entry['name']] +=1
    
   
              
        
    
# correct for nan values in data, get into json object form
df_metadata['production_companies'] = df_metadata['production_companies'].fillna('[]')
df_metadata['production_companies'] = df_metadata['production_companies'].apply(lambda x: str(x).replace('\'', '"'))
df_metadata['production_companies'].apply(tester)

print(f'Most Common Production Companies:')
print(sorted(ProdCompanies.items(), key= lambda item: item[1], reverse=True)[:10])

# create categories for major film studios
df_metadata['Warner Bros'] = 0
df_metadata['MGM'] = 0
df_metadata['Paramount Pictures'] = 0
df_metadata['Universal'] = 0
df_metadata['20th Century Fox'] = 0
df_metadata['Disney Pixar'] = 0
df_metadata['Columbia Pictures'] = 0

df_metadata['Number of Production Orgs'] = 0


def fillProductions(row):
    try:
        compList = json.loads(row['production_companies'])
    except JSONDecodeError as e:
        return row
    
    countr = 0
    for entry in compList: 
        countr += 1
        
        if entry['name'] == 'Warner Bros.':
            row['Warner Bros'] += 1
        elif entry['name'] == 'Metro-Goldwyn-Mayer (MGM)':
            row['MGM'] += 1
        elif entry['name'] == 'Paramount Pictures':
            row['Paramount Pictures'] += 1
        elif entry['name'] == 'Universal Pictures':
            row['Universal'] += 1
        elif entry['name'] == 'Twentieth Century Fox Film Corporation':
            row['20th Century Fox'] += 1
        elif entry['name'] in ['Columbia Pictures Corporation','Columbia Pictures']:
            row['Columbia Pictures'] += 1
        elif entry['name'] in ['Walt Disney Productions', 'Walt Disney Animation Studios', 'Pixar Animation Studios']:
            row['Disney Pixar'] += 1
      
    row['Num_Production_Orgs'] = countr
    
    return row

df_metadata = df_metadata.apply(fillProductions, axis=1)

# drop companies field
df_metadata = df_metadata.drop(['production_companies'], axis=1)
df_metadata[['id', 'Num_Production_Orgs', 'Warner Bros','MGM', 'Paramount Pictures', 'Universal', '20th Century Fox','Columbia Pictures', 'Disney Pixar' ]].head()


Most Common Production Companies:
[('Warner Bros.', 1243), ('Metro-Goldwyn-Mayer (MGM)', 1045), ('Paramount Pictures', 1000), ('Universal Pictures', 826), ('Twentieth Century Fox Film Corporation', 821), ('Columbia Pictures Corporation', 446), ('Columbia Pictures', 429), ('Canal+', 390), ('RKO Radio Pictures', 290), ('United Artists', 277)]


,id,Num_Production_Orgs,Warner Bros,MGM,Paramount Pictures,Universal,20th Century Fox,Columbia Pictures,Disney Pixar
0,862,1.0,0,0,0,0,0,0,1
1,8844,3.0,0,0,0,0,0,0,0
2,15602,2.0,1,0,0,0,0,0,0
3,31357,1.0,0,0,0,0,1,0,0
4,11862,2.0,0,0,0,0,0,0,0


In [58]:
# extract out the release year and month 
df_metadata['year'] = 0
df_metadata['month'] = 0

def extractDate(row):
    
    if type(row['release_date']) == float:
        row['year'] = 0
        row['month'] = 0
        return row
        
    dateString = row['release_date']
    dateList = dateString.split('-')
    row['year'] = int(dateList[0])
    row['month'] = int(dateList[1])
     
    return row

df_metadata = df_metadata.apply(extractDate, axis=1)

# drop date column
df_metadata = df_metadata.drop(['release_date'], axis=1)

df_metadata[['id', 'year', 'month']].head()



,id,year,month
0,862,1995,10
1,8844,1995,12
2,15602,1995,12
3,31357,1995,12
4,11862,1995,2


In [59]:
# Extract Number of Production companies

df_metadata['Num_Prod_Countries'] = 0
df_metadata['Made_in_US'] = 0

def ExtractCountries(row):
    if row['production_countries'] == 'nan':
        return row
    
    countriesString = row['production_countries'].replace('\'', '"')
    countriesDict = json.loads(countriesString)
    countr = 0
    for entry in countriesDict:
        countr +=1
        if entry['iso_3166_1'] == 'US':
            row['Made_in_US'] = 1
        
    row['Num_Prod_Countries'] = countr
    return row
   
        

df_metadata['production_countries'] = df_metadata['production_countries'].apply(lambda x: str(x).replace('D\'Ivoire', 'DIvoire'))
df_metadata['production_countries'] = df_metadata['production_countries'].apply(lambda x: str(x).replace('ople\'s Dem', 'oples Dem'))
df_metadata = df_metadata.apply(ExtractCountries, axis=1)

df_metadata = df_metadata.drop(['production_countries'], axis=1)

df_metadata[['id', 'Made_in_US', 'Num_Prod_Countries']].head()




,id,Made_in_US,Num_Prod_Countries
0,862,1,1
1,8844,1,1
2,15602,1,1
3,31357,1,1
4,11862,1,1


In [60]:
# extract number of spoken languages

def getLangs(val):
    if str(val) == 'nan':
        return 0
   
    val = str(val).replace('\'', '"')
    val = val.replace('\\x', '')
    valList = json.loads(val)
    countr = 0
    for entry in valList:
        countr +=1
        
    return countr
    
    
df_metadata['spoken_languages'] = df_metadata['spoken_languages'].apply(getLangs)

df_metadata[['id', 'spoken_languages']].head()


print(df_metadata.columns)

Index(['20th Century Fox', 'Action', 'Adventure', 'Animation',
       'Columbia Pictures', 'Comedy', 'Crime', 'Disney Pixar', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'MGM',
       'Music', 'Mystery', 'Num_Production_Orgs', 'Number of Production Orgs',
       'Paramount Pictures', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'Universal', 'War', 'Warner Bros', 'Western', 'adult',
       'belongs_to_collection', 'budget', 'has_homepage', 'id', 'imdb_id',
       'original_language', 'popularity', 'revenue', 'runtime',
       'spoken_languages', 'vote_average', 'vote_count', 'year', 'month',
       'Num_Prod_Countries', 'Made_in_US'],
      dtype='object')


In [61]:
# Pull in next Data file
df_credits = pd.read_csv("./data/credits.csv", dtype=str)

df_metadata = df_metadata.drop_duplicates(subset=['id'])
df_credits = df_credits.drop_duplicates(subset=['id'])

df_credits = df_credits.astype({'id' : int})
df = df_metadata.merge(df_credits, on='id', how='inner')

df.head()

print(df[])



,20th Century Fox,Action,Adventure,Animation,Columbia Pictures,Comedy,Crime,Disney Pixar,Documentary,Drama,...,runtime,spoken_languages,vote_average,vote_count,year,month,Num_Prod_Countries,Made_in_US,cast,crew
0,0,0,0,1,0,1,0,1,0,0,...,81.0,1,7.7,5415,1995,10,1,1,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,0,0,1,0,0,0,0,0,0,0,...,104.0,2,6.9,2413,1995,12,1,1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,0,0,0,0,1,0,0,0,0,...,101.0,1,6.5,92,1995,12,1,1,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,1,0,0,0,0,1,0,0,0,1,...,127.0,1,6.1,34,1995,12,1,1,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,0,0,0,0,1,0,0,0,0,...,106.0,1,5.7,173,1995,2,1,1,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
